# **AI TECH INSTITUTE** · *Intermediate AI & Data Science*
### Week 04 · Notebook 06 – Sampling Theory & Law of Large Numbers
**Instructor:** Amir Charkhi  |  **Goal:** Master the foundations of statistical inference through sampling.

> Format: short theory → quick practice → build understanding → mini-challenges.


---
## Learning Objectives
- Understand population vs sample concepts
- Master different sampling techniques
- Prove the Law of Large Numbers empirically
- Apply Central Limit Theorem in practice

## 1. Population vs Sample: The Foundation
Understanding why we sample and what it means for inference.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Create a "population" - imagine all users of a social media platform
POPULATION_SIZE = 1000000  # 1 million users

# True population parameters (usually unknown in real life!)
TRUE_MEAN_AGE = 28
TRUE_STD_AGE = 8
TRUE_PREMIUM_RATE = 0.15  # 15% are premium users

# Generate population
print("Creating population of 1 million users...")
population_ages = np.random.normal(TRUE_MEAN_AGE, TRUE_STD_AGE, POPULATION_SIZE)
population_ages = np.clip(population_ages, 13, 80)  # Realistic age bounds
population_premium = np.random.binomial(1, TRUE_PREMIUM_RATE, POPULATION_SIZE)

print(f"Population created!")
print(f"True population mean age: {population_ages.mean():.2f}")
print(f"True population std age: {population_ages.std():.2f}")
print(f"True premium rate: {population_premium.mean():.2%}")

In [ ]:
# Take samples of different sizes
sample_sizes = [10, 50, 100, 500, 1000, 5000]
samples = {}

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, n in enumerate(sample_sizes):
    # Take a random sample
    sample_indices = np.random.choice(POPULATION_SIZE, n, replace=False)
    sample_ages = population_ages[sample_indices]
    samples[n] = sample_ages
    
    # Calculate sample statistics
    sample_mean = sample_ages.mean()
    sample_std = sample_ages.std(ddof=1)  # Using sample std (n-1)
    
    # Visualize
    axes[idx].hist(sample_ages, bins=30, density=True, alpha=0.7, color='blue', edgecolor='black')
    axes[idx].axvline(sample_mean, color='red', linestyle='--', linewidth=2, label=f'Sample mean: {sample_mean:.2f}')
    axes[idx].axvline(TRUE_MEAN_AGE, color='green', linestyle='--', linewidth=2, label=f'True mean: {TRUE_MEAN_AGE}')
    axes[idx].set_title(f'Sample Size: {n}\nError: {abs(sample_mean - TRUE_MEAN_AGE):.2f} years')
    axes[idx].set_xlabel('Age')
    axes[idx].set_ylabel('Density')
    axes[idx].legend(fontsize=8)
    axes[idx].set_xlim(10, 50)

plt.suptitle('How Sample Size Affects Estimation Accuracy', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("\n📊 Key Insight: Larger samples give estimates closer to true population values!")

**Exercise 1 – Sample Bias Detection (easy)**  
Compare random vs biased sampling and see the effect.


In [ ]:
# Your turn
# Take two samples of size 1000:
# 1. Random sample
# 2. Biased sample (only ages < 25)
# Compare their means to the true population mean


<details>
<summary><b>Solution</b></summary>

```python
# Random sample
random_sample = np.random.choice(population_ages, 1000, replace=False)
random_mean = random_sample.mean()

# Biased sample (younger users only)
young_indices = np.where(population_ages < 25)[0]
biased_sample = population_ages[np.random.choice(young_indices, 1000, replace=False)]
biased_mean = biased_sample.mean()

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Random sample
ax1.hist(random_sample, bins=30, alpha=0.7, color='green', edgecolor='black')
ax1.axvline(random_mean, color='red', linestyle='--', linewidth=2, label=f'Sample mean: {random_mean:.2f}')
ax1.axvline(TRUE_MEAN_AGE, color='blue', linestyle='--', linewidth=2, label=f'True mean: {TRUE_MEAN_AGE}')
ax1.set_title(f'Random Sample\nBias: {random_mean - TRUE_MEAN_AGE:.2f} years')
ax1.set_xlabel('Age')
ax1.legend()

# Biased sample
ax2.hist(biased_sample, bins=30, alpha=0.7, color='red', edgecolor='black')
ax2.axvline(biased_mean, color='red', linestyle='--', linewidth=2, label=f'Sample mean: {biased_mean:.2f}')
ax2.axvline(TRUE_MEAN_AGE, color='blue', linestyle='--', linewidth=2, label=f'True mean: {TRUE_MEAN_AGE}')
ax2.set_title(f'Biased Sample (Age < 25 only)\nBias: {biased_mean - TRUE_MEAN_AGE:.2f} years')
ax2.set_xlabel('Age')
ax2.legend()

plt.suptitle('Random vs Biased Sampling', fontsize=14)
plt.tight_layout()
plt.show()

print(f"\n⚠️ Biased sampling leads to incorrect estimates!")
print(f"Random sample error: {abs(random_mean - TRUE_MEAN_AGE):.2f} years")
print(f"Biased sample error: {abs(biased_mean - TRUE_MEAN_AGE):.2f} years")
```
</details>

## 2. The Law of Large Numbers in Action
Watch the magic happen as sample size increases!

In [ ]:
# Demonstrate Law of Large Numbers - Coin Flips
def law_of_large_numbers_demo(true_probability=0.5, max_flips=10000):
    """Show how sample proportion converges to true probability"""
    
    # Simulate coin flips
    flips = np.random.binomial(1, true_probability, max_flips)
    
    # Calculate running average
    running_average = np.cumsum(flips) / np.arange(1, max_flips + 1)
    
    # Plot convergence
    plt.figure(figsize=(12, 6))
    
    # Main plot
    plt.subplot(1, 2, 1)
    plt.plot(running_average, alpha=0.7, linewidth=1)
    plt.axhline(y=true_probability, color='red', linestyle='--', linewidth=2, label=f'True probability: {true_probability}')
    plt.fill_between(range(max_flips), 
                     running_average - 2*np.sqrt(true_probability*(1-true_probability)/np.arange(1, max_flips+1)),
                     running_average + 2*np.sqrt(true_probability*(1-true_probability)/np.arange(1, max_flips+1)),
                     alpha=0.2, color='gray', label='95% CI')
    plt.xlabel('Number of Flips')
    plt.ylabel('Proportion of Heads')
    plt.title('Law of Large Numbers: Coin Flips')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Zoom in on convergence
    plt.subplot(1, 2, 2)
    plt.plot(running_average[-1000:], alpha=0.7, linewidth=1)
    plt.axhline(y=true_probability, color='red', linestyle='--', linewidth=2)
    plt.xlabel('Last 1000 Flips')
    plt.ylabel('Proportion of Heads')
    plt.title('Zoomed View: Convergence')
    plt.ylim(true_probability - 0.05, true_probability + 0.05)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print convergence statistics
    print(f"After {max_flips:,} flips:")
    print(f"Sample proportion: {running_average[-1]:.4f}")
    print(f"True probability: {true_probability:.4f}")
    print(f"Error: {abs(running_average[-1] - true_probability):.4f}")

# Run the demo
law_of_large_numbers_demo(true_probability=0.5, max_flips=10000)

In [ ]:
# LLN with different probabilities
probabilities = [0.1, 0.3, 0.5, 0.7, 0.9]
n_trials = 5000

fig, axes = plt.subplots(1, len(probabilities), figsize=(20, 4))

for idx, p in enumerate(probabilities):
    # Simulate
    outcomes = np.random.binomial(1, p, n_trials)
    running_avg = np.cumsum(outcomes) / np.arange(1, n_trials + 1)
    
    # Plot
    axes[idx].plot(running_avg, alpha=0.7, linewidth=1, color='blue')
    axes[idx].axhline(y=p, color='red', linestyle='--', linewidth=2)
    axes[idx].set_title(f'p = {p}')
    axes[idx].set_xlabel('Trials')
    axes[idx].set_ylabel('Sample Proportion')
    axes[idx].set_ylim(0, 1)
    axes[idx].grid(True, alpha=0.3)
    
    # Add final value
    axes[idx].text(n_trials*0.7, 0.1, f'Final: {running_avg[-1]:.3f}', 
                  bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.suptitle('Law of Large Numbers Works for Any Probability!', fontsize=14, y=1.05)
plt.tight_layout()
plt.show()

**Exercise 2 – Casino Simulation (medium)**  
Simulate a casino game and show the house always wins in the long run.


In [ ]:
# Your turn
# Simulate roulette: 18 red, 18 black, 2 green (0, 00)
# Bet on red: win $1 if red, lose $1 otherwise
# Show cumulative profit/loss over 10000 spins


<details>
<summary><b>Solution</b></summary>

```python
# Roulette simulation
n_spins = 10000

# Probabilities
p_red = 18/38  # American roulette
p_not_red = 20/38

# Expected value per bet
expected_value = p_red * 1 + p_not_red * (-1)
print(f"Expected value per $1 bet: ${expected_value:.4f}")
print(f"House edge: {-expected_value*100:.2f}%\n")

# Simulate spins
outcomes = np.random.choice([1, -1], size=n_spins, p=[p_red, p_not_red])
cumulative_profit = np.cumsum(outcomes)

# Expected profit line
expected_profit = expected_value * np.arange(1, n_spins + 1)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Cumulative profit
ax1.plot(cumulative_profit, alpha=0.7, label='Actual profit/loss')
ax1.plot(expected_profit, 'r--', linewidth=2, label='Expected (LLN)')
ax1.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax1.fill_between(range(n_spins), 0, cumulative_profit, 
                 where=(cumulative_profit >= 0), alpha=0.3, color='green', label='Profit')
ax1.fill_between(range(n_spins), 0, cumulative_profit, 
                 where=(cumulative_profit < 0), alpha=0.3, color='red', label='Loss')
ax1.set_xlabel('Number of Spins')
ax1.set_ylabel('Cumulative Profit ($)')
ax1.set_title('Casino Always Wins in the Long Run')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Average profit per spin
avg_profit = cumulative_profit / np.arange(1, n_spins + 1)
ax2.plot(avg_profit, alpha=0.7)
ax2.axhline(y=expected_value, color='red', linestyle='--', linewidth=2, 
           label=f'Expected: ${expected_value:.4f}')
ax2.set_xlabel('Number of Spins')
ax2.set_ylabel('Average Profit per Spin ($)')
ax2.set_title('Convergence to Expected Value')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nAfter {n_spins:,} spins:")
print(f"Total profit/loss: ${cumulative_profit[-1]:,.2f}")
print(f"Average per spin: ${avg_profit[-1]:.4f}")
print(f"\n🎰 The house edge guarantees casino profits over time!")
```
</details>

## 3. Sampling Methods & Their Effects

In [ ]:
# Create a diverse population for sampling demonstration
np.random.seed(42)
n_population = 100000

# Create population with different segments
population_df = pd.DataFrame({
    'user_id': range(n_population),
    'age': np.concatenate([
        np.random.normal(22, 3, n_population//4),   # Young segment
        np.random.normal(35, 5, n_population//4),   # Middle segment
        np.random.normal(50, 7, n_population//4),   # Older segment
        np.random.normal(65, 5, n_population//4)    # Senior segment
    ]),
    'income': np.concatenate([
        np.random.lognormal(10, 0.5, n_population//4),  # Low income
        np.random.lognormal(10.8, 0.4, n_population//4), # Medium income
        np.random.lognormal(11.2, 0.3, n_population//4), # High income
        np.random.lognormal(10.5, 0.6, n_population//4)  # Variable income
    ]),
    'segment': np.repeat(['Young', 'Middle', 'Older', 'Senior'], n_population//4)
})

# Add region based on some pattern
population_df['region'] = np.random.choice(['North', 'South', 'East', 'West'], 
                                          n_population, p=[0.2, 0.3, 0.25, 0.25])

print("Population created with segments:")
print(population_df.groupby('segment')['age'].agg(['mean', 'count']))

In [ ]:
def compare_sampling_methods(population_df, sample_size=1000):
    """Compare different sampling methods"""
    
    results = {}
    
    # 1. Simple Random Sampling
    simple_random = population_df.sample(n=sample_size, random_state=42)
    results['Simple Random'] = simple_random
    
    # 2. Systematic Sampling (every kth element)
    k = len(population_df) // sample_size
    systematic = population_df.iloc[::k][:sample_size]
    results['Systematic'] = systematic
    
    # 3. Stratified Sampling (proportional to segments)
    stratified = population_df.groupby('segment', group_keys=False).apply(
        lambda x: x.sample(int(len(x) * sample_size / len(population_df)), random_state=42)
    )
    results['Stratified'] = stratified
    
    # 4. Cluster Sampling (select some regions entirely)
    selected_regions = np.random.choice(population_df['region'].unique(), 2, replace=False)
    cluster = population_df[population_df['region'].isin(selected_regions)].sample(n=sample_size, random_state=42)
    results['Cluster'] = cluster
    
    return results

# Compare methods
sample_size = 1000
sampling_results = compare_sampling_methods(population_df, sample_size)

# Visualize differences
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for idx, (method, sample) in enumerate(sampling_results.items()):
    # Age distribution
    axes[0, idx].hist(sample['age'], bins=30, alpha=0.7, density=True, edgecolor='black')
    axes[0, idx].axvline(sample['age'].mean(), color='red', linestyle='--', linewidth=2)
    axes[0, idx].set_title(f'{method}\nMean Age: {sample["age"].mean():.1f}')
    axes[0, idx].set_xlabel('Age')
    axes[0, idx].set_ylabel('Density')
    
    # Segment proportions
    segment_props = sample['segment'].value_counts(normalize=True)
    axes[1, idx].bar(segment_props.index, segment_props.values, alpha=0.7)
    axes[1, idx].set_title(f'Segment Distribution')
    axes[1, idx].set_xlabel('Segment')
    axes[1, idx].set_ylabel('Proportion')
    axes[1, idx].tick_params(axis='x', rotation=45)

plt.suptitle('Comparison of Sampling Methods', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# Statistical comparison
print("\nSampling Method Comparison:")
print("="*60)
population_mean_age = population_df['age'].mean()
population_mean_income = population_df['income'].mean()

for method, sample in sampling_results.items():
    age_error = abs(sample['age'].mean() - population_mean_age)
    income_error = abs(sample['income'].mean() - population_mean_income)
    print(f"{method:15s} | Age Error: {age_error:.2f} | Income Error: ${income_error:.0f}")

## 4. Central Limit Theorem: The Magic of Sampling Distributions

In [ ]:
def demonstrate_clt(distribution_type='exponential', sample_size=30, n_samples=10000):
    """Show that sample means are normally distributed regardless of population distribution"""
    
    # Generate population based on distribution type
    if distribution_type == 'exponential':
        population_generator = lambda n: np.random.exponential(scale=5, size=n)
        title = 'Exponential Population'
    elif distribution_type == 'uniform':
        population_generator = lambda n: np.random.uniform(0, 10, size=n)
        title = 'Uniform Population'
    elif distribution_type == 'bimodal':
        def bimodal_generator(n):
            return np.concatenate([
                np.random.normal(3, 1, n//2),
                np.random.normal(8, 1, n//2)
            ])
        population_generator = bimodal_generator
        title = 'Bimodal Population'
    
    # Generate many samples and calculate their means
    sample_means = []
    for _ in range(n_samples):
        sample = population_generator(sample_size)
        sample_means.append(sample.mean())
    
    sample_means = np.array(sample_means)
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original distribution
    original = population_generator(10000)
    axes[0].hist(original, bins=50, density=True, alpha=0.7, color='blue', edgecolor='black')
    axes[0].set_title(f'{title}\n(Original Distribution)')
    axes[0].set_xlabel('Value')
    axes[0].set_ylabel('Density')
    
    # Sampling distribution of means
    axes[1].hist(sample_means, bins=50, density=True, alpha=0.7, color='green', edgecolor='black')
    
    # Overlay normal distribution
    mu = sample_means.mean()
    sigma = sample_means.std()
    x = np.linspace(sample_means.min(), sample_means.max(), 100)
    axes[1].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal fit')
    axes[1].set_title(f'Distribution of Sample Means\n(n={sample_size}, {n_samples} samples)')
    axes[1].set_xlabel('Sample Mean')
    axes[1].set_ylabel('Density')
    axes[1].legend()
    
    # Q-Q plot to verify normality
    stats.probplot(sample_means, dist="norm", plot=axes[2])
    axes[2].set_title('Q-Q Plot\n(Testing Normality of Sample Means)')
    
    plt.suptitle(f'Central Limit Theorem: {title} → Normal Distribution of Means', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Normality test
    _, p_value = stats.shapiro(sample_means[:5000])  # Shapiro test limited to 5000
    print(f"Shapiro-Wilk test for normality of sample means:")
    print(f"p-value: {p_value:.4f}")
    if p_value > 0.05:
        print("✅ Sample means are normally distributed (CLT confirmed!)")
    else:
        print("⚠️ Sample means may not be perfectly normal (try larger sample size)")

# Demonstrate with different distributions
demonstrate_clt('exponential', sample_size=30)

In [ ]:
# Show CLT with different sample sizes
sample_sizes_clt = [2, 5, 10, 30, 50, 100]
n_simulations = 5000

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Use a heavily skewed distribution (exponential)
for idx, n in enumerate(sample_sizes_clt):
    # Generate sample means
    sample_means = [np.random.exponential(scale=5, size=n).mean() 
                   for _ in range(n_simulations)]
    
    # Plot histogram
    axes[idx].hist(sample_means, bins=40, density=True, alpha=0.7, 
                  color='purple', edgecolor='black')
    
    # Overlay theoretical normal
    theoretical_mean = 5  # Mean of exponential with scale=5
    theoretical_std = 5 / np.sqrt(n)  # Standard error
    x = np.linspace(min(sample_means), max(sample_means), 100)
    axes[idx].plot(x, stats.norm.pdf(x, theoretical_mean, theoretical_std), 
                  'r-', linewidth=2, label='Theoretical Normal')
    
    axes[idx].set_title(f'n = {n}')
    axes[idx].set_xlabel('Sample Mean')
    axes[idx].set_ylabel('Density')
    axes[idx].legend(fontsize=8)
    
    # Add skewness value
    skew = stats.skew(sample_means)
    axes[idx].text(0.7, 0.9, f'Skew: {skew:.3f}', 
                  transform=axes[idx].transAxes,
                  bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.suptitle('CLT: Sample Means Become Normal as n Increases\n(Even from Exponential Population!)', 
            fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("📊 Key Insight: Even with n=30, sample means are approximately normal!")
print("This is why n=30 is often cited as the 'magic number' for CLT.")

**Exercise 3 – Bootstrap Confidence Intervals (medium)**  
Use bootstrap sampling to estimate confidence intervals.


In [ ]:
# Your turn
# Given a sample of 100 incomes, use bootstrap to:
# 1. Estimate the sampling distribution of the mean
# 2. Calculate 95% confidence interval
# sample_incomes = np.random.lognormal(10, 1, 100)


<details>
<summary><b>Solution</b></summary>

```python
# Original sample
np.random.seed(42)
sample_incomes = np.random.lognormal(10, 1, 100)
original_mean = sample_incomes.mean()

# Bootstrap
n_bootstrap = 10000
bootstrap_means = []

for _ in range(n_bootstrap):
    # Resample with replacement
    bootstrap_sample = np.random.choice(sample_incomes, size=len(sample_incomes), replace=True)
    bootstrap_means.append(bootstrap_sample.mean())

bootstrap_means = np.array(bootstrap_means)

# Calculate confidence interval
ci_lower = np.percentile(bootstrap_means, 2.5)
ci_upper = np.percentile(bootstrap_means, 97.5)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Original sample
ax1.hist(sample_incomes, bins=30, alpha=0.7, color='blue', edgecolor='black')
ax1.axvline(original_mean, color='red', linestyle='--', linewidth=2, 
           label=f'Sample mean: ${original_mean:,.0f}')
ax1.set_xlabel('Income ($)')
ax1.set_ylabel('Frequency')
ax1.set_title('Original Sample (n=100)')
ax1.legend()

# Bootstrap distribution
ax2.hist(bootstrap_means, bins=50, alpha=0.7, color='green', edgecolor='black')
ax2.axvline(original_mean, color='red', linestyle='--', linewidth=2, label='Original mean')
ax2.axvline(ci_lower, color='orange', linestyle='--', linewidth=2)
ax2.axvline(ci_upper, color='orange', linestyle='--', linewidth=2)
ax2.fill_betweenx([0, ax2.get_ylim()[1]], ci_lower, ci_upper, 
                  alpha=0.3, color='orange', label='95% CI')
ax2.set_xlabel('Bootstrap Sample Mean ($)')
ax2.set_ylabel('Frequency')
ax2.set_title(f'Bootstrap Distribution ({n_bootstrap} resamples)')
ax2.legend()

plt.suptitle('Bootstrap Confidence Interval Estimation', fontsize=14)
plt.tight_layout()
plt.show()

print(f"Bootstrap Results:")
print(f"Original sample mean: ${original_mean:,.2f}")
print(f"Bootstrap mean: ${bootstrap_means.mean():,.2f}")
print(f"Bootstrap std error: ${bootstrap_means.std():,.2f}")
print(f"\n95% Confidence Interval: [${ci_lower:,.2f}, ${ci_upper:,.2f}]")
print(f"\n✅ We're 95% confident the true population mean is in this interval!")
```
</details>

## 5. Standard Error and Sample Size Calculations

In [ ]:
# How sample size affects precision
def sample_size_precision_demo():
    """Show relationship between sample size and standard error"""
    
    # True population parameters
    true_mean = 100
    true_std = 15
    
    sample_sizes = np.arange(10, 1000, 10)
    
    # Theoretical standard error
    theoretical_se = true_std / np.sqrt(sample_sizes)
    
    # Empirical standard error (via simulation)
    empirical_se = []
    for n in sample_sizes:
        sample_means = [np.random.normal(true_mean, true_std, n).mean() 
                       for _ in range(500)]
        empirical_se.append(np.std(sample_means))
    
    # Margin of error for 95% CI
    margin_of_error = 1.96 * theoretical_se
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Standard Error vs Sample Size
    axes[0].plot(sample_sizes, theoretical_se, 'b-', linewidth=2, label='Theoretical')
    axes[0].scatter(sample_sizes[::5], empirical_se[::5], alpha=0.5, s=20, 
                   color='red', label='Empirical')
    axes[0].set_xlabel('Sample Size')
    axes[0].set_ylabel('Standard Error')
    axes[0].set_title('Standard Error Decreases with √n')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Margin of Error
    axes[1].plot(sample_sizes, margin_of_error, 'g-', linewidth=2)
    axes[1].axhline(y=1, color='red', linestyle='--', alpha=0.7, label='Target: ±1')
    axes[1].fill_between(sample_sizes, 0, margin_of_error, 
                        where=(margin_of_error <= 1), alpha=0.3, color='green')
    axes[1].set_xlabel('Sample Size')
    axes[1].set_ylabel('Margin of Error (95% CI)')
    axes[1].set_title('Precision Improves with Sample Size')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Required sample size for different margins
    desired_margins = [5, 3, 2, 1, 0.5]
    required_n = [(1.96 * true_std / margin) ** 2 for margin in desired_margins]
    
    axes[2].bar(range(len(desired_margins)), required_n, alpha=0.7, color='purple')
    axes[2].set_xticks(range(len(desired_margins)))
    axes[2].set_xticklabels([f'±{m}' for m in desired_margins])
    axes[2].set_xlabel('Desired Margin of Error')
    axes[2].set_ylabel('Required Sample Size')
    axes[2].set_title('Sample Size Requirements')
    axes[2].set_yscale('log')
    
    # Add values on bars
    for i, n in enumerate(required_n):
        axes[2].text(i, n, f'{int(n):,}', ha='center', va='bottom')
    
    plt.suptitle('The Precision-Sample Size Relationship', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Print key insights
    print("Key Insights:")
    print("="*50)
    print(f"• To halve the margin of error, you need 4× the sample size")
    print(f"• Diminishing returns: Going from n=100 to n=400 reduces SE by 50%")
    print(f"• But going from n=400 to n=700 only reduces SE by 15%")

sample_size_precision_demo()

**Exercise 4 – Monte Carlo Integration (hard)**  
Use random sampling to estimate π (pi).


In [ ]:
# Your turn
# Use Monte Carlo method to estimate π:
# 1. Generate random points in a square
# 2. Check if they fall inside a circle
# 3. Use the ratio to estimate π
# Show convergence as n increases


<details>
<summary><b>Solution</b></summary>

```python
def monte_carlo_pi(n_points=10000, visualize=True):
    """Estimate π using Monte Carlo sampling"""
    
    # Generate random points in [0,1] x [0,1]
    x = np.random.uniform(0, 1, n_points)
    y = np.random.uniform(0, 1, n_points)
    
    # Check if inside quarter circle (radius = 1)
    inside_circle = (x**2 + y**2) <= 1
    
    # Estimate π
    # Area of quarter circle = π/4
    # Area of square = 1
    # Ratio = π/4
    pi_estimates = np.cumsum(inside_circle) / np.arange(1, n_points + 1) * 4
    
    if visualize:
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # Scatter plot of points
        sample_points = min(5000, n_points)
        axes[0].scatter(x[:sample_points], y[:sample_points], 
                       c=inside_circle[:sample_points], 
                       s=1, alpha=0.5, cmap='RdBu')
        
        # Draw quarter circle
        theta = np.linspace(0, np.pi/2, 100)
        axes[0].plot(np.cos(theta), np.sin(theta), 'r-', linewidth=2)
        axes[0].set_xlim(0, 1)
        axes[0].set_ylim(0, 1)
        axes[0].set_aspect('equal')
        axes[0].set_title(f'Monte Carlo Sampling\n(First {sample_points:,} points)')
        axes[0].set_xlabel('x')
        axes[0].set_ylabel('y')
        
        # Convergence plot
        axes[1].plot(pi_estimates, alpha=0.7, linewidth=1)
        axes[1].axhline(y=np.pi, color='red', linestyle='--', linewidth=2, label=f'True π = {np.pi:.6f}')
        axes[1].set_xlabel('Number of Points')
        axes[1].set_ylabel('Estimate of π')
        axes[1].set_title('Convergence to π')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        # Error plot
        errors = np.abs(pi_estimates - np.pi)
        axes[2].loglog(range(1, n_points + 1), errors, alpha=0.5, linewidth=1)
        axes[2].set_xlabel('Number of Points (log scale)')
        axes[2].set_ylabel('|Error| (log scale)')
        axes[2].set_title('Error Decreases with √n')
        axes[2].grid(True, alpha=0.3)
        
        # Add 1/√n reference line
        n_range = np.arange(10, n_points, 100)
        axes[2].plot(n_range, 1/np.sqrt(n_range), 'r--', alpha=0.7, label='1/√n')
        axes[2].legend()
        
        plt.suptitle(f'Monte Carlo Estimation of π', fontsize=14, y=1.02)
        plt.tight_layout()
        plt.show()
    
    return pi_estimates[-1]

# Run simulation
n_points = 100000
pi_estimate = monte_carlo_pi(n_points)

print(f"\nResults after {n_points:,} points:")
print(f"Estimated π: {pi_estimate:.6f}")
print(f"True π:      {np.pi:.6f}")
print(f"Error:       {abs(pi_estimate - np.pi):.6f}")
print(f"Error %:     {abs(pi_estimate - np.pi)/np.pi*100:.3f}%")

# Show convergence rate
print("\nConvergence Rate:")
for n in [100, 1000, 10000, 100000, 1000000]:
    estimate = monte_carlo_pi(n, visualize=False)
    error = abs(estimate - np.pi)
    print(f"n = {n:8,}: π ≈ {estimate:.5f}, error = {error:.5f}")
```
</details>

## 6. Real-World Application: A/B Test Sample Size

In [ ]:
def ab_test_sample_size_calculator(baseline_rate, minimum_detectable_effect, 
                                  alpha=0.05, power=0.8):
    """Calculate required sample size for A/B test"""
    
    from statsmodels.stats.proportion import proportion_effectsize
    from statsmodels.stats.power import zt_ind_solve_power
    
    # Calculate effect size
    effect_size = proportion_effectsize(baseline_rate, 
                                       baseline_rate + minimum_detectable_effect)
    
    # Calculate required sample size
    n_required = zt_ind_solve_power(effect_size=effect_size,
                                    alpha=alpha,
                                    power=power,
                                    ratio=1)
    
    return int(np.ceil(n_required))

# Interactive sample size exploration
baseline_rates = [0.01, 0.05, 0.10, 0.20, 0.30]
relative_improvements = [0.05, 0.10, 0.20, 0.30, 0.50]  # Relative changes

# Create heatmap of required sample sizes
sample_size_matrix = np.zeros((len(baseline_rates), len(relative_improvements)))

for i, baseline in enumerate(baseline_rates):
    for j, rel_improvement in enumerate(relative_improvements):
        abs_improvement = baseline * rel_improvement
        n = ab_test_sample_size_calculator(baseline, abs_improvement)
        sample_size_matrix[i, j] = n

# Visualize
plt.figure(figsize=(10, 8))
sns.heatmap(sample_size_matrix, annot=True, fmt='.0f', cmap='YlOrRd',
            xticklabels=[f'+{int(r*100)}%' for r in relative_improvements],
            yticklabels=[f'{int(b*100)}%' for b in baseline_rates])
plt.xlabel('Relative Improvement to Detect')
plt.ylabel('Baseline Conversion Rate')
plt.title('Required Sample Size per Group for A/B Test\n(α=0.05, Power=0.8)')
plt.tight_layout()
plt.show()

print("Key Insights:")
print("="*50)
print("• Detecting small improvements requires huge samples")
print("• Lower baseline rates need larger samples")
print("• Doubling the effect size quarters the required sample")

## 7. Mini-Challenges
- **M1 (easy):** Simulate dice rolls and show convergence to expected value
- **M2 (medium):** Implement reservoir sampling for streaming data
- **M3 (hard):** Create a function to detect Simpson's Paradox in grouped data

In [ ]:
# Your turn - try the challenges!


<details>
<summary><b>Solutions</b></summary>

```python
# M1 - Dice rolls convergence
n_rolls = 10000
dice_rolls = np.random.randint(1, 7, n_rolls)
running_average = np.cumsum(dice_rolls) / np.arange(1, n_rolls + 1)

plt.figure(figsize=(10, 5))
plt.plot(running_average, alpha=0.7)
plt.axhline(y=3.5, color='red', linestyle='--', linewidth=2, label='Expected: 3.5')
plt.xlabel('Number of Rolls')
plt.ylabel('Average Value')
plt.title('Law of Large Numbers: Fair Dice')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()
print(f"After {n_rolls:,} rolls: {running_average[-1]:.4f}")

# M2 - Reservoir sampling
def reservoir_sampling(stream, k):
    """Select k items uniformly from stream of unknown length"""
    reservoir = []
    
    for i, item in enumerate(stream):
        if i < k:
            reservoir.append(item)
        else:
            j = np.random.randint(0, i + 1)
            if j < k:
                reservoir[j] = item
    
    return reservoir

# Test reservoir sampling
stream = range(1000000)  # Large stream
sample = reservoir_sampling(stream, 100)
print(f"Reservoir sample mean: {np.mean(sample):.1f}")
print(f"True mean: {(1000000-1)/2:.1f}")

# M3 - Simpson's Paradox detector
def detect_simpsons_paradox(df, group_col, x_col, y_col):
    """Detect if correlation reverses when grouped"""
    
    # Overall correlation
    overall_corr = df[x_col].corr(df[y_col])
    
    # Group correlations
    group_corrs = []
    for group in df[group_col].unique():
        group_data = df[df[group_col] == group]
        if len(group_data) > 2:
            corr = group_data[x_col].corr(group_data[y_col])
            group_corrs.append(corr)
    
    # Check for paradox
    if overall_corr > 0:
        paradox = all(c < 0 for c in group_corrs)
    else:
        paradox = all(c > 0 for c in group_corrs)
    
    print(f"Overall correlation: {overall_corr:.3f}")
    print(f"Group correlations: {[f'{c:.3f}' for c in group_corrs]}")
    
    if paradox:
        print("⚠️ Simpson's Paradox detected!")
    else:
        print("✅ No paradox detected")
    
    return paradox

# Create example with Simpson's Paradox
np.random.seed(42)
data = pd.DataFrame({
    'group': np.repeat(['A', 'B', 'C'], 100),
    'x': np.concatenate([np.random.normal(10, 2, 100),
                        np.random.normal(20, 2, 100),
                        np.random.normal(30, 2, 100)]),
    'y': np.concatenate([np.random.normal(30, 2, 100) - np.random.normal(10, 2, 100)*0.5,
                        np.random.normal(20, 2, 100) - np.random.normal(20, 2, 100)*0.5,
                        np.random.normal(10, 2, 100) - np.random.normal(30, 2, 100)*0.5])
})

detect_simpsons_paradox(data, 'group', 'x', 'y')
```
</details>

## Wrap-Up & Next Steps
✅ You understand the relationship between populations and samples  
✅ You've proven the Law of Large Numbers empirically  
✅ You've seen the Central Limit Theorem in action  
✅ You know how sample size affects precision  
✅ You can apply sampling theory to real problems  

**Key Takeaways:**
- Larger samples → Better estimates (LLN)
- Sample means → Normal distribution (CLT)  
- Random sampling → Unbiased inference
- Sample size ∝ 1/error² (quadratic relationship)

**Week 4 Complete!** You now have a deep understanding of statistical foundations, distributions, EDA techniques, and the mathematical basis for why statistics works!
